In [ ]:
from FAdo.reex import *
from os.path import commonprefix

In [ ]:
class Node:
    
    def __init__(self,expr,string,t):
        self.type  = type(expr)
        self.path  = string
        self.exp   = expr
        self.tree  = t
        self.right = None
        self.left  = None
        self.first = set()
        self.last  = set()
        self.lsa   = None
        self.supfirst = None
        self.suplast = None
        self.FollowAfter = set()
        self.colors = set()
        if self.type is position:
            t.atoms.add(self)
        if string == '1':
            self.parent = None
        else:
            self.parent = t.NT.get(string[:-1])
        
    def get_root(self):
        return self.tree.root
    
    def get_brother(self):
        if self.path[len(self.path)-1]==1:
            return self.parent.right
        else:
            return self.parent.left
    
    def reflexive(self,n):
        if n is None:
            return False
        if commonprefix([self.path,n.path])==n.path: #LCA(self,n)
            return True
        return False
        
    # boolean that states if an atom is the first from the right child of a concatenation
    def SupFirst(self):
        father = self.parent
        if father is not None and father.type is concat:
            if father.right==self and not father.left.exp.ewp():
                return True
        return False

    # boolean that states if an atom is the last from the left child of a concatenation
    def SupLast(self):
        father = self.parent
        if father is not None and father.type is concat:
            if father.left==self and not father.right.exp.ewp():
                return True
        return False
    
    # pointer to the first(right child) of a concatenation
    def pSupFirst(self):
        if self.supfirst is not None:
            return self.supfirst
        self.supfirst = None
        if self.SupFirst() or self.path=="1":
            self.supfirst = self
        else:
            self.supfirst = self.parent.pSupFirst()
        return self.supfirst

    # pointer to the last(left child) of a concatenation
    def pSupLast(self):
        if self.suplast is not None:
            return self.suplast
        if self.SupLast() or self.path=="1":
            self.suplast = self
        else:
            self.suplast = self.parent.pSupLast()
        return self.suplast
    
    def follow(self,p):
        node1 = self.tree.NT.get(commonprefix([p.path,self.path]))  # LCA(p,self)
        if node1 is None:
            return False
        if node1.type is concat:
            if self.follow_concat(p,node1):
                return True
        node = node1.LSA()
        if node is not None:
            if self.follow_star(p,node):
                return True
        return False
    
    def follow_concat(self,p,lca):
        if self in lca.right.First() and p in lca.left.Last():
            return True
        return False
                
    def follow_star(self,p,lsa):
        if self in lsa.First() and p in lsa.Last():
            return True
        return False
    
    # list of first atoms from a reg exp using lemma 2.3
    def First(self):
        if self.first != set():
            return self.first
        for node in self.tree.atoms:
            if isFirst(self,node):
                self.first.add(node)
        return self.first
    
    # list of last atoms from a reg exp using lemma 2.3
    def Last(self):
        if self.last != set():
            return self.last
        for node in self.tree.atoms:
            if isLast(self,node):
                self.last.add(node)
        return self.last
    
    def LSA(self):
        if self.lsa is not None:
            return self.lsa
        if self.type is star:
            self.lsa = self
        elif len(self.path) == 1:
            self.lsa = None
        else:
            self.lsa = self.parent.LSA()
        return self.lsa
    
    def FirstPos(self,ta):
        k = set()
        for x in ta:
            if x in self.First():
                k.add(x)
        return k
    
    def follow_after(self):
        s = set()
        for p in self.Last():
            for q in self.tree.atoms:
                if q.follow(p) and not q.reflexive(self):
                    s.add(q)
        self.FollowAfter = s
        return s
    
    

# lemma 2.3
def isFirst(n,p):
    k = p.pSupFirst()
    if k is not None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    return False

def isLast(n,p):
    k = p.pSupLast()
    if k is not None:
        if p.reflexive(n) and n.reflexive(k):
            return True
    return False